# Introduction
Hello, from the new project. I will create an AI assistant. This AI assistant converts the Python code to C++ code. I will use LLM models from Claude and OpenAI. 

In [1]:
# Python Library
import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [2]:
# import api key
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')

In [3]:
openai = OpenAI()
claude = anthropic.Anthropic()

OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"

In [4]:
system_message = "You are an assistant that reimplements Python code in high performance C++. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [5]:
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt

In [6]:
def messages_for(python):
    return [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_prompt_for(python)}
    ]

In [7]:
def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```", "")
    with open("optimized_working.cpp", "w") as f:
        f.write(code)

In [8]:
# This function convert the python code to C++ code with gpt-4o
def optimize_gpt(python):
    stream = openai.chat.completions.create(
        model = OPENAI_MODEL,
        messages = messages_for(python),
        stream = True
    )

    reply = ""

    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end = "", flush = True)
    write_output(reply)

In [9]:
# This function convert the python code to C++ code with claude-3-5-sonnet
def optimize_claude(python):
    result = claude.messages.stream(
        model = CLAUDE_MODEL,
        max_tokens = 2000,
        system = system_message,
        messages = [{'role': 'user', 'content': user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            print(text, end="", flush=True)

    write_output(reply)

In [10]:
# example Python code
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [13]:
# I am using gpt-4o-mini model for convert code!
optimize_gpt(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(int iterations, int param1, int param2) {
    double result = 1.0;
    for (int i = 1; i <= iterations; ++i) {
        double j = static_cast<double>(i * param1 - param2);
        result -= (1.0 / j);
        j = static_cast<double>(i * param1 + param2);
        result += (1.0 / j);
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    
    double result = calculate(100000000, 4, 1) * 4;
    
    auto end_time = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> execution_time = end_time - start_time;
    
    std::cout << "Result: " << std::fixed << std::setprecision(12) << result << "\n";
    std::cout << "Execution Time: " << std::fixed << std::setprecision(6) << execution_time.count() << " seconds\n";
    
    return 0;
}
```

In [15]:
optimize_claude(pi)

#include <iostream>
ip>clude <ioman
 <chrono>

 long iterations, long long param1, long long param2) {
1.0;double result = 
 reduction(-:result)llel for
for (long long i = 1; i <= iterations; ++i) {
 j1 = i * param1 - param2;
 j2 = i * param1 + param2;
result -= (1.0 / j1);
result += (1.0 / j2);
    }
    return result;
}

 {t main()
_time = std::chrono::high_resolution_clock::now();
    
(100'000'000, 4, 1) * 4;ulate
    
d_time = std::chrono::high_resolution_clock::now();
 duration = std::chrono::duration_cast<std::chrono::microseconds>(end_time - start_time);
    
< std::fixed << std::setprecision(12);
 << "Result: " << result << std::endl;
d::cout << "Execution Time: " << duration.count() / 1e6 << " seconds" << std::endl;
    
return 0;
}

In [16]:
python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [17]:
optimize_gpt(python_hard)

```cpp
#include <iostream>
#include <vector>
#include <chrono>
#include <limits>

using namespace std;
using ll = long long;

class LCG {
public:
    LCG(ll seed, ll a = 1664525, ll c = 1013904223, ll m = 1ULL << 32) 
        : value(seed), a(a), c(c), m(m) {}

    ll next() {
        value = (a * value + c) % m;
        return value;
    }

private:
    ll value;
    const ll a, c, m;
};

ll max_subarray_sum(int n, ll seed, int min_val, int max_val) {
    LCG lcg_gen(seed);
    vector<int> random_numbers(n);
    for (int i = 0; i < n; ++i) {
        random_numbers[i] = lcg_gen.next() % (max_val - min_val + 1) + min_val;
    }

    ll max_sum = numeric_limits<ll>::min();
    for (int i = 0; i < n; ++i) {
        ll current_sum = 0;
        for (int j = i; j < n; ++j) {
            current_sum += random_numbers[j];
            if (current_sum > max_sum) {
                max_sum = current_sum;
            }
        }
    }
    return max_sum;
}

ll total_max_subarray_sum(int n, ll initi

In [18]:
optimize_claude(python_hard)

#include <iostream>
#include <vector>
ono>lude <chr
>include <limits
#include <algorithm>
#include <iomanip>

 namespace std;
ono;g namespace std::chr

class LCG {
private:
;   uint64_t value
 = 1664525;int64_t a
1013904223;int64_t c = 
_t m = 1ULL << 32;

public:
uint64_t seed) : value(seed) {}

 {  uint64_t next()
 c) % m;value = (a * value +
        return value;
    }
};

64_t max_subarray_sum(int n, uint64_t seed, int min_val, int max_val) {
CG lcg(seed);
<int64_t> random_numbers(n);
 = 0; i < n; ++i) {
 = lcg.next() % (max_val - min_val + 1) + min_val;
    }

 max_sum = numeric_limits<int64_t>::min();
64_t current_sum = 0;
_sum = 0;_t min

 (int i = 0; i < n; ++i) {
 += random_numbers[i];
_sum = max(max_sum, current_sum - min_sum);
, current_sum); = min(min_sum
    }

return max_sum;
}

_t total_max_subarray_sum(int n, uint64_t initial_seed, int min_val, int max_val) {
_sum = 0;_t total
G lcg(initial_seed);
 = 0; i < 20; ++i) {
d = lcg.next();t see
sum += max_subarray_sum(n, seed

# Conclusion
In this project, I used two model for convert to code. My AI assistant converted the python code to c++ code. Thank you for taking a look at my project. I will continue to share projects. If you want to be informed in advance, you can follow me from the links below.

[LinkedIn](https://www.linkedin.com/in/ihsancenkiz/)<br>
[GitHub](https://github.com/ihsncnkz)<br>
[Kaggle](https://www.kaggle.com/ihsncnkz)<br>